In [ ]:
import mne
import sys

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

import time
from IPython.display import clear_output


In [ ]:
def run_ica(s: int, moment, n_components = None, auto_pick = False):

    # Load epoch
    epoch_fname = dirs['epoch'] + '/epoch_' + moment + '_s' + str(s) + '.fif'
    epoch = mne.read_epochs(epoch_fname, preload=True)

    # High-pass filter
    epoch.filter(1,None)

    # Fit ICA
    ica = mne.preprocessing.ICA(n_components, max_iter="auto").fit(epoch)
    
    # EOG correlates
    ica.exclude, scores = ica.find_bads_eog(epoch)

    if auto_pick: # Exclude max
        ica.plot_scores(scores)
        ica.plot_components(scores.argmax())
        ica.exclude = [int(scores.argmax())]

    else: # Pick components
        ica.plot_components()
        ica.plot_scores(scores)

        bad_comp = input('Components to exclude (_,_): ').split(sep=',')
        ica.exclude = [int(c) for c in bad_comp]

    # Save
    ica_fname = dirs['ica'] + '/ica_s' + str(s) + '.fif'
    ica.save(ica_fname, overwrite = True)

    return ica

In [ ]:
# Run all subjects

for s in subjects[11:]:
    print('Running subject ' + str(s))
    ica = run_ica(s, moment = 'enc1', auto_pick = False)
    clear_output(wait=False)

In [ ]:
# Run single subject

s = 12; n_components = .999

ica = run_ica(s, moment = 'enc1', n_components = n_components, auto_pick = False)